In [1]:
pip install pandas numpy scikit-learn joblib boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.7 MB/s eta 0:00:00


In [2]:
!pip install moto==4.1.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import joblib
import boto3
from moto import mock_s3

In [4]:
# Generate synthetic smart home energy data
def generate_synthetic_data(n_samples=1000):
    np.random.seed(42)
    data = {
        'temperature': np.random.uniform(10, 35, n_samples),
        'humidity': np.random.uniform(30, 80, n_samples),
        'time_of_day': np.random.uniform(0, 24, n_samples),
        'day_of_week': np.random.randint(0, 7, n_samples),
        'occupancy': np.random.randint(0, 5, n_samples),
        'appliance_usage': np.random.uniform(0, 10, n_samples),
        'energy_consumption': None
    }

    # Create realistic relationship between features and energy consumption
    data['energy_consumption'] = (
        0.5 * data['temperature'] +
        0.3 * data['humidity'] +
        0.2 * np.sin(data['time_of_day'] * np.pi / 12) +
        0.1 * data['day_of_week'] +
        0.4 * data['occupancy'] +
        0.6 * data['appliance_usage'] +
        np.random.normal(0, 0.5, n_samples)
    )

    return pd.DataFrame(data)

# Generate and prepare data
df = generate_synthetic_data()
X = df.drop('energy_consumption', axis=1)
y = df['energy_consumption']

In [5]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=42)

In [6]:
# Evaluate the model
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)
print(f"Training R² score: {train_score:.4f}")
print(f"Testing R² score: {test_score:.4f}")

# Save the model and scaler
joblib.dump(model, 'energy_optimizer_model.joblib')
joblib.dump(scaler, 'scaler.joblib')

Training R² score: 0.9955
Testing R² score: 0.9727


['scaler.joblib']

In [7]:
# AWS S3 upload
with mock_s3():
    s3 = boto3.client('s3')
    s3.create_bucket(Bucket='smart-home-energy-models')
    s3.upload_file('energy_optimizer_model.joblib', 'smart-home-energy-models', 'energy_optimizer_model.joblib')
    s3.upload_file('scaler.joblib', 'smart-home-energy-models', 'scaler.joblib')